Final version for basic knowledge graph --> refactoring notebooks

In [1]:
# pip install pyvis

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from pyvis import network as net
from pyvis.network import Network

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [3]:
def features_eng(df, version):
    """
    
    """
    if version == 'clustering':
        df = df.drop(columns=['dist2','TransactionID'])
    elif version == 'anomaly':
        df = df.drop(columns=['dist2','customer_id','TransactionID'])
    elif version == 'network':
        df = df.drop(columns=['dist2'])
    df = df.rename(columns={'id_31':'browser'})
    df['P_emaildomain'] = df['P_emaildomain'].mask(df['P_emaildomain']=='gmail','gmail.com')
    df['R_emaildomain'] = df['R_emaildomain'].mask(df['R_emaildomain']=='gmail','gmail.com')
    df['id_30'] = df['id_30'].replace(" ","_",regex=True)
    df['id_30'] = df['id_30'].str.replace(".","_",regex=False)
    df['browser'] = df['browser'].mask(df['browser'].str.contains('SM') | df['browser'].str.contains('ZTE'),'other')
    df['browser'] = df['browser'].astype(str)
    df['browser_enc'] = 'other'
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('ie'),'ie')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('safari'),'safari')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('edge'),'edge')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('firefox'),'firefox')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('android'),'android')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('Android'),'android')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('chrome'),'chrome')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('opera'),'opera')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('google'),'chrome')
    df['browser_enc'] = df['browser_enc'].mask(df['browser'].str.contains('samsung'),'android')
    df['DeviceType'] = df['DeviceType'].mask(df['browser']== 'ie 11.0 for tablet','tablet')
    df = df.drop(columns='browser')
    df['device_info'] = df['DeviceInfo'].replace("-","_",regex=True)
    df['device_info2'] = df['device_info'].replace(" ","_",regex=True)
    df['device_info'] = df['DeviceInfo'].replace("-","_",regex=True)
    df['device_info2'] = df['device_info'].replace(" ","_",regex=True)
    df['device_info3'] = df['device_info2'].str.split("_").str[0]
    df['device_info3'] = df['device_info3'].str.lower()
    df['device_info_v4'] = 'other'
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='windows') | (df['device_info3']=='microsoft') | (df['device_info3']=='trident/7.0')),'windows')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='ios') | (df['device_info3']=='iphone')),'ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='blade','blade')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='lenovo','lenovo')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='redmi','redmi')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='pixel','pixel')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='android','android')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='alcatel','alcatel')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='nokia','nokia')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='asus','asus')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='oneplus','oneplus')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='zte','zte')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='macos','ios')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='hisense','hisense')
    df['device_info_v4'] = df['device_info_v4'].mask(df['device_info3']=='linux','linux')
    df['device_info_v4'] = df['device_info_v4'].mask((df['device_info3'].str.contains("lg") | (df['device_info3'].str.contains("nexus"))),'lg')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3'].str.contains("huawei")) | (df['device_info3'].str.contains("hi6210sft"))),'huawei')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='motog3') |(df['device_info3']=='moto')),'moto')
    df['device_info_v4'] = df['device_info_v4'].mask(((df['device_info3']=='sm') |(df['device_info3']=='samsung')),'samsung')
    df = df.drop(columns=['device_info','device_info2','device_info3','DeviceInfo'])
    return df

In [4]:
def building_relation_dict(rels):
    """
    
    """
    dt = dict()
    for i in range(len(rels)):
        dt[i] = dict()
        dt[i]['source'] = rels[i][0]
        dt[i]['target'] = rels[i][1]
        dt[i]['rel'] = rels[i][2]
    return dt

In [5]:
def building_adj_list(dt,df):
    """
    
    """
    db = []
    for i in df.index:
        # select one row 
        row = df.loc[df.index==i]
        # create empty dataframe to be filled with info taken from the row selected
        gp = pd.DataFrame(columns=['source','target','rel'])
        # for loop for taking index and name of the dict of the relationship
        for y in dt.keys():
            for z in dt[y].keys():
                # for the rel value, we need to extract the relatioship value from the dict not from the row of the dataframe! 
                if z == 'rel':
                    gp.at[y,z] = dt[y][z]
                else:
                    gp.at[y,z] = row[dt[y][z]][i]
        db.append(gp)
    final = pd.concat(db)
    final = final.reset_index().drop(columns='index')
    final = final.drop_duplicates()
    return final

In [6]:
def building_network(final,final2,df,num):
    """
    
    """
    net = Network(directed=True,height='900px', width='100%',notebook=False,cdn_resources='in_line',select_menu=True, filter_menu=True)
    nodes = list(pd.unique(final[['source','target']].values.ravel('K')))
    nodes2 = list(pd.unique(final2[['source','target']].values.ravel('K')))
    tra = list(df['TransactionID'].unique())
    pdr = list(df['ProductCD'].unique())
    # add nodes
    for e in nodes:
        if e in pdr:
            net.add_node(e,color='blue')
        elif e in tra:
            val = df[df['TransactionID']==e]['TransactionAmt'].values[0]
            if e in nodes2:
                net.add_node(e,color='red',value=val)
            else:
                net.add_node(e,color='green',value=val)
        elif e in nodes2:
            net.add_node(e,color='red')
            
        else:
            net.add_node(e,color='green')
    # add edges
    for i in final.index:
        row = final.loc[final.index==i]
        net.add_edge(row['source'].item(),row['target'].item(),title=row['rel'].item(),label=row['rel'].item())
#     net.show_buttons(filter_=['physics'])
    net.set_options("""const options = {
  "physics": {
    "repulsion": {
      "springLength": 400,
      "springConstant": 0.1,
      "nodeDistance": 400
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  }
}""")
    return net.show("kg_v29_{}.html".format(num))

# 1.0 Data retrieval

In [7]:
df = pd.read_pickle("../../../data_lake/output_prod/df_under.pkl")

# 2.0 Data preparation

In [8]:
df = features_eng(df,'network')

In [9]:
df = df[['customer_id','TransactionID','TransactionAmt','DeviceType','device_info_v4','browser_enc','ProductCD','isFraud']]

In [10]:
df = df.dropna(axis=0)

In [11]:
df = df.sample(frac=0.0005,random_state=4)

In [12]:
df['isFraud'].value_counts()

0    63
1     7
Name: isFraud, dtype: int64

In [13]:
for i in df:
    if i != 'TransactionAmt':
        df[i] = df[i].astype(str)

# 3.0 Modeling

In [14]:
rel1 = ['customer_id','TransactionID','has done']
rel2 = ['TransactionID','ProductCD','buying']
rel3 = ['ProductCD','DeviceType','by']
rel4 = ['TransactionID','DeviceType','with']
rel5 = ['TransactionID','browser_enc','on']

In [15]:
rels = [rel1,rel2,rel3,rel4,rel5]

In [16]:
dt = building_relation_dict(rels)

In [17]:
final = building_adj_list(dt,df)

Extraction of the fraud only 

In [18]:
df2 = df[df['isFraud']=='1']

In [19]:
rel1 = ['customer_id','TransactionID','has done']
rels = [rel1]

In [20]:
dt2 = building_relation_dict(rels)

In [21]:
final2 = building_adj_list(dt2,df2)

In [22]:
building_network(final,final2,df,10)

PermissionError: [Errno 1] Operation not permitted: 'bindings'

For Networkx:

In [23]:
g = nx.from_pandas_edgelist(final,'source','target',edge_attr=True,edge_key='rel',create_using=nx.DiGraph())

In [25]:
print(nx.info(g))

Name: 
Type: DiGraph
Number of nodes: 148
Number of edges: 288
Average in degree:   1.9459
Average out degree:   1.9459


and so on...